In [ ]:
# librerias
!pip install -qU langchain langchain_openai langchain-core langchain-community langgraph psycopg[binary,pool]==3.2.6



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.1/467.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 

In [ ]:
import os
from datetime import datetime
from typing import Dict, List, Optional, Literal
from pydantic import BaseModel, Field
import pandas as pd
from pathlib import Path

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [ ]:

from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine

with open("/content/drive/MyDrive/api_key_openAI.txt") as archivo:
  apikey = archivo.read()
os.environ["OPENAI_API_KEY"] = apikey

In [ ]:
# Rutas de archivos
RUTA_EXCEL_ALUMNOS = "/content/drive/MyDrive/Ingresantes2025_EXMED.xlsx"
RUTA_EXCEL_EXAMENES = "/content/drive/MyDrive/Examenes_Registrados.xlsx"


Gestionando datos en el archivo excel

In [ ]:
class GestorExcel:
    """Gestor de archivos Excel para alumnos y exámenes"""

    def __init__(self, ruta_alumnos: str, ruta_examenes: str):
        self.ruta_alumnos = ruta_alumnos
        self.ruta_examenes = ruta_examenes
        self._crear_archivo_examenes_si_no_existe()

    def _crear_archivo_examenes_si_no_existe(self):
        """Crea el archivo de exámenes si no existe"""
        if not Path(self.ruta_examenes).exists():
            df_examenes = pd.DataFrame(columns=[
                'CODIGO_MATRICULA', 'TIPO_EXAMEN', 'FECHA_EXAMEN',
                'RESULTADO', 'ESTADO', 'OBSERVACIONES', 'RESPONSABLE'
            ])
            df_examenes.to_excel(self.ruta_examenes, index=False)

    def cargar_alumnos(self) -> pd.DataFrame:
        """Carga el archivo de alumnos"""
        try:
            df = pd.read_excel(self.ruta_alumnos)
            df.columns = df.columns.str.strip()
            return df
        except Exception as e:
            raise Exception(f"Error al cargar archivo de alumnos: {str(e)}")

    def cargar_examenes(self) -> pd.DataFrame:
        """Carga el archivo de exámenes"""
        try:
            return pd.read_excel(self.ruta_examenes)
        except Exception as e:
            return pd.DataFrame(columns=[
                'CODIGO_MATRICULA', 'TIPO_EXAMEN', 'FECHA_EXAMEN',
                'RESULTADO', 'ESTADO', 'OBSERVACIONES', 'RESPONSABLE'
            ])

    def guardar_examenes(self, df: pd.DataFrame):
        """Guarda el DataFrame de exámenes"""
        df.to_excel(self.ruta_examenes, index=False)

    def buscar_alumno(self, filtro_tipo: str, valor: str) -> List[Dict]:
        """Busca alumnos según filtro"""
        df = self.cargar_alumnos()

        if filtro_tipo == "codigo":
            resultado = df[df['CODIGO MATRICULA'].astype(str) == str(valor)]
        elif filtro_tipo == "documento":
            resultado = df[df['NUMERO DOCUMENTO'].astype(str) == str(valor)]
        elif filtro_tipo == "apellido":
            mask = (df['APELLIDO PATERNO'].str.contains(valor, case=False, na=False) |
                   df['APELLIDO MATERNO'].str.contains(valor, case=False, na=False))
            resultado = df[mask]
        else:
            return []

        return resultado.to_dict('records')

    def obtener_examenes_alumno(self, codigo_matricula: str) -> List[Dict]:
        """Obtiene exámenes de un alumno"""
        df_examenes = self.cargar_examenes()
        examenes = df_examenes[
            df_examenes['CODIGO_MATRICULA'].astype(str) == str(codigo_matricula)
        ]
        return examenes.to_dict('records')

    def registrar_examen(self, codigo_matricula: str, tipo_examen: str,
                        resultado: str, estado: str, observaciones: str,
                        responsable: str) -> Dict:
        """Registra o actualiza un examen"""
        df_examenes = self.cargar_examenes()

        # Buscar si ya existe un examen de este tipo para este alumno
        mask = ((df_examenes['CODIGO_MATRICULA'].astype(str) == str(codigo_matricula)) &
                (df_examenes['TIPO_EXAMEN'] == tipo_examen))

        nuevo_registro = {
            'CODIGO_MATRICULA': codigo_matricula,
            'TIPO_EXAMEN': tipo_examen,
            'FECHA_EXAMEN': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'RESULTADO': resultado,
            'ESTADO': estado,
            'OBSERVACIONES': observaciones,
            'RESPONSABLE': responsable
        }

        if mask.any():
            # Actualizar registro existente
            for col, val in nuevo_registro.items():
                df_examenes.loc[mask, col] = val
            accion = "actualizado"
        else:
            # Agregar nuevo registro
            df_examenes = pd.concat([df_examenes, pd.DataFrame([nuevo_registro])],
                                   ignore_index=True)
            accion = "registrado"

        self.guardar_examenes(df_examenes)
        return {"accion": accion, "registro": nuevo_registro}

gestor = GestorExcel(RUTA_EXCEL_ALUMNOS, RUTA_EXCEL_EXAMENES)


## Buscar al alumno por código, documento o apellido

In [ ]:
class ConsultaAlumnoInput(BaseModel):
    filtro_tipo: Literal["codigo", "documento", "apellido"] = Field(
        description="Tipo de filtro: 'codigo' para código de matrícula, 'documento' para número de documento, 'apellido' para buscar por apellido"
    )
    valor_filtro: str = Field(description="Valor a buscar")


In [ ]:
@tool("consultar_alumno", args_schema=ConsultaAlumnoInput)
def consultar_alumno(filtro_tipo: str, valor_filtro: str) -> Dict:
    """
    Busca información de alumnos en el archivo Excel.
    Permite buscar por código de matrícula, número de documento o apellido.
    Retorna datos personales y estado de exámenes médicos.
    """
    try:
        alumnos = gestor.buscar_alumno(filtro_tipo, valor_filtro)

        if not alumnos:
            return {"error": "No se encontró ningún alumno con ese criterio"}

        # Para cada alumno, obtener sus exámenes
        resultados = []
        for alumno in alumnos:
            codigo = alumno.get('CODIGO MATRICULA')
            examenes = gestor.obtener_examenes_alumno(codigo)

            # Convertir fecha si existe
            if 'FECHA NACIMIENTO' in alumno and pd.notna(alumno['FECHA NACIMIENTO']):
                alumno['FECHA NACIMIENTO'] = str(alumno['FECHA NACIMIENTO'])

            # Calcular progreso
            examenes_requeridos = ['laboratorio', 'psicologico', 'odontologia', 'rayos_x']
            examenes_completados = [e for e in examenes if e.get('ESTADO') == 'completado']

            resultados.append({
                "datos_personales": alumno,
                "examenes_realizados": examenes,
                "examenes_completados": len(examenes_completados),
                "examenes_pendientes": len(examenes_requeridos) - len(examenes_completados)
            })

        return {
            "exito": True,
            "cantidad_encontrados": len(resultados),
            "alumnos": resultados
        }

    except Exception as e:
        return {"error": f"Error en la búsqueda: {str(e)}"}

## 2. Registra o actualiza el resultado de un examen para un alumno.

In [ ]:
class RegistroExamenInput(BaseModel):
    codigo_matricula: str = Field(description="Código de matrícula del alumno")
    tipo_examen: Literal["laboratorio", "psicologico", "odontologia", "rayos_x"] = Field(
        description="Tipo de examen: laboratorio, psicologico, odontologia, rayos_x"
    )
    resultado: str = Field(description="Resultado del examen (descripción completa)")
    estado: Literal["completado", "requiere_seguimiento"] = Field(
        description="Estado: completado o requiere_seguimiento"
    )
    observaciones: str = Field(description="Observaciones adicionales", default="")
    responsable: str = Field(description="Nombre del responsable que registra")

@tool("registrar_examen", args_schema=RegistroExamenInput)
def registrar_examen(codigo_matricula: str, tipo_examen: str, resultado: str,
                     estado: str, observaciones: str = "", responsable: str = "") -> Dict:
    """
    Registra el resultado de un examen médico en Excel.
    Valida que el alumno exista y actualiza o crea el registro.
    Si el examen ya existía, lo actualiza con la nueva información.
    """
    try:
        # Verificar que el alumno existe
        alumnos = gestor.buscar_alumno("codigo", codigo_matricula)

        if not alumnos:
            return {"error": f"No se encontró alumno con código {codigo_matricula}"}

        alumno = alumnos[0]

        # Registrar el examen
        registro_info = gestor.registrar_examen(
            codigo_matricula, tipo_examen, resultado,
            estado, observaciones, responsable
        )

        # Traducir tipo de examen
        tipos_examen_es = {
            "laboratorio": "Análisis de Laboratorio",
            "psicologico": "Evaluación Psicológica",
            "odontologia": "Examen Odontológico",
            "rayos_x": "Rayos X de Tórax"
        }

        return {
            "exito": True,
            "mensaje": f"Examen {registro_info['accion']} para {alumno.get('NOMBRES', '')} {alumno.get('APELLIDO PATERNO', '')}",
            "tipo_examen": tipos_examen_es.get(tipo_examen, tipo_examen),
            "estado": estado,
            "requiere_alerta": estado == "requiere_seguimiento"
        }

    except Exception as e:
        return {"error": f"Error al registrar examen: {str(e)}"}

## 3. Muestra el progreso detallado (completados, pendientes) de un alumno.

In [ ]:
@tool
def obtener_estado_examenes(codigo_matricula: str) -> Dict:
    """
    Obtiene el estado completo de todos los exámenes de un alumno.
    Muestra cuáles están completados, pendientes o requieren seguimiento.
    Calcula el porcentaje de avance del proceso médico.
    """
    try:
        # Verificar que el alumno existe
        alumnos = gestor.buscar_alumno("codigo", codigo_matricula)

        if not alumnos:
            return {"error": f"No se encontró alumno con código {codigo_matricula}"}

        alumno = alumnos[0]
        examenes = gestor.obtener_examenes_alumno(codigo_matricula)

        # Tipos de examen requeridos
        examenes_requeridos = {
            "laboratorio": "Análisis de Laboratorio",
            "psicologico": "Evaluación Psicológica",
            "odontologia": "Examen Odontológico",
            "rayos_x": "Rayos X de Tórax"
        }

        # Organizar exámenes por tipo
        examenes_por_tipo = {}
        for examen in examenes:
            tipo = examen['TIPO_EXAMEN']
            examenes_por_tipo[tipo] = examen

        # Identificar pendientes
        examenes_pendientes = [
            {"tipo": tipo, "nombre": nombre}
            for tipo, nombre in examenes_requeridos.items()
            if tipo not in examenes_por_tipo
        ]

        # Identificar los que requieren seguimiento
        requieren_seguimiento = [
            {"tipo": exam['TIPO_EXAMEN'],
             "observaciones": exam.get('OBSERVACIONES', '')}
            for exam in examenes
            if exam.get('ESTADO') == 'requiere_seguimiento'
        ]

        # Calcular progreso
        completados = len([e for e in examenes if e.get('ESTADO') == 'completado'])
        progreso = (completados / len(examenes_requeridos)) * 100

        return {
            "exito": True,
            "alumno": f"{alumno.get('NOMBRES', '')} {alumno.get('APELLIDO PATERNO', '')} {alumno.get('APELLIDO MATERNO', '')}",
            "codigo_matricula": codigo_matricula,
            "escuela": alumno.get('NOMBRE ESCUELA', ''),
            "progreso_porcentaje": round(progreso, 1),
            "examenes_completados": completados,
            "total_examenes": len(examenes_requeridos),
            "detalle_examenes": examenes_por_tipo,
            "examenes_pendientes": examenes_pendientes,
            "requieren_seguimiento": requieren_seguimiento,
            "proceso_completo": len(examenes_pendientes) == 0 and len(requieren_seguimiento) == 0
        }

    except Exception as e:
        return {"error": f"Error al consultar estado: {str(e)}"}

## Genera una lista de todos los alumnos a los que les falta algún examen.

In [ ]:
class ListaPendientesInput(BaseModel):
    tipo_examen: Optional[str] = Field(
        description="Filtrar por tipo de examen específico (opcional)",
        default=None
    )
    escuela: Optional[str] = Field(
        description="Filtrar por nombre de escuela (opcional)",
        default=None
    )

@tool("listar_alumnos_pendientes", args_schema=ListaPendientesInput)
def listar_alumnos_pendientes(tipo_examen: Optional[str] = None,
                               escuela: Optional[str] = None) -> Dict:
    """
    Lista todos los alumnos que tienen exámenes médicos pendientes.
    Puede filtrar por tipo de examen específico o por escuela.
    Útil para planificar jornadas médicas.
    """
    try:
        df_alumnos = gestor.cargar_alumnos()
        df_examenes = gestor.cargar_examenes()

        resultados = []

        for _, alumno in df_alumnos.iterrows():
            codigo = alumno['CODIGO MATRICULA']

            # Filtro por escuela si se especificó
            if escuela and escuela.lower() not in str(alumno.get('NOMBRE ESCUELA', '')).lower():
                continue

            # Obtener exámenes del alumno
            examenes_alumno = df_examenes[
                df_examenes['CODIGO_MATRICULA'].astype(str) == str(codigo)
            ]

            # Identificar exámenes completados
            completados = examenes_alumno[
                examenes_alumno['ESTADO'] == 'completado'
            ]['TIPO_EXAMEN'].tolist()

            # Calcular pendientes
            todos = ['laboratorio', 'psicologico', 'odontologia', 'rayos_x']
            pendientes = [e for e in todos if e not in completados]

            # Si se filtró por tipo de examen, verificar
            if tipo_examen and tipo_examen not in pendientes:
                continue

            if pendientes:  # Solo incluir si tiene pendientes
                resultados.append({
                    "codigo_matricula": str(codigo),
                    "nombre_completo": f"{alumno['NOMBRES']} {alumno['APELLIDO PATERNO']} {alumno['APELLIDO MATERNO']}",
                    "escuela": alumno.get('NOMBRE ESCUELA', ''),
                    "examenes_pendientes": pendientes,
                    "cantidad_pendientes": len(pendientes)
                })

        return {
            "exito": True,
            "total_alumnos_pendientes": len(resultados),
            "filtro_tipo_examen": tipo_examen,
            "filtro_escuela": escuela,
            "alumnos": resultados[:20]
        }

    except Exception as e:
        return {"error": f"Error al listar pendientes: {str(e)}"}

## 5. Crea estadísticas generales de avance (ej. % de alumnos completados).

In [ ]:
class ReporteInput(BaseModel):
    tipo_reporte: Literal["general", "por_escuela"] = Field(
        description="Tipo de reporte: general o por_escuela"
    )
    parametro: Optional[str] = Field(
        description="Nombre de escuela si el tipo es 'por_escuela'",
        default=None
    )

@tool("generar_reporte", args_schema=ReporteInput)
def generar_reporte(tipo_reporte: str, parametro: Optional[str] = None) -> Dict:
    """
    Genera reportes estadísticos sobre el proceso de exámenes médicos.
    Incluye métricas de avance, distribución por escuela y estado general.
    """
    try:
        df_alumnos = gestor.cargar_alumnos()
        df_examenes = gestor.cargar_examenes()

        total_alumnos = len(df_alumnos)

        if tipo_reporte == "general":
            # Calcular alumnos con proceso completo
            alumnos_completos = 0
            for codigo in df_alumnos['CODIGO MATRICULA']:
                examenes = df_examenes[
                    df_examenes['CODIGO_MATRICULA'].astype(str) == str(codigo)
                ]
                completados = examenes[examenes['ESTADO'] == 'completado']
                if len(completados) == 4:
                    alumnos_completos += 1

            # Alumnos con seguimiento
            codigos_seguimiento = df_examenes[
                df_examenes['ESTADO'] == 'requiere_seguimiento'
            ]['CODIGO_MATRICULA'].unique()

            # Exámenes por tipo
            examenes_por_tipo = df_examenes[
                df_examenes['ESTADO'] == 'completado'
            ].groupby('TIPO_EXAMEN').size().to_dict()

            reporte = {
                "tipo": "Reporte General",
                "fecha": datetime.now().strftime("%Y-%m-%d %H:%M"),
                "total_alumnos": total_alumnos,
                "proceso_completo": alumnos_completos,
                "porcentaje_completado": round((alumnos_completos / total_alumnos * 100) if total_alumnos > 0 else 0, 1),
                "requieren_seguimiento": len(codigos_seguimiento),
                "examenes_realizados_por_tipo": examenes_por_tipo
            }

        elif tipo_reporte == "por_escuela":
            if not parametro:
                return {"error": "Se requiere nombre de escuela"}

            # Filtrar alumnos de la escuela
            alumnos_escuela = df_alumnos[
                df_alumnos['NOMBRE ESCUELA'].str.contains(parametro, case=False, na=False)
            ]

            total_escuela = len(alumnos_escuela)

            # Calcular completados
            completos_escuela = 0
            for codigo in alumnos_escuela['CODIGO MATRICULA']:
                examenes = df_examenes[
                    df_examenes['CODIGO_MATRICULA'].astype(str) == str(codigo)
                ]
                completados = examenes[examenes['ESTADO'] == 'completado']
                if len(completados) == 4:
                    completos_escuela += 1

            reporte = {
                "tipo": "Reporte por Escuela",
                "fecha": datetime.now().strftime("%Y-%m-%d %H:%M"),
                "escuela": parametro,
                "total_alumnos": total_escuela,
                "proceso_completo": completos_escuela,
                "porcentaje_completado": round((completos_escuela / total_escuela * 100) if total_escuela > 0 else 0, 1)
            }

        return {
            "exito": True,
            "reporte": reporte
        }

    except Exception as e:
        return {"error": f"Error al generar reporte: {str(e)}"}

## CONFIGURACIÓN DE AGENTES


In [ ]:
# AGENTE 1: COORDINADOR GENERAL
llm_coordinador = ChatOpenAI(temperature=0.3, model="gpt-4o-mini")
memory_coordinador = MemorySaver()

prompt_coordinador = ChatPromptTemplate.from_messages([
    ("system", """Eres el Agente Coordinador del Sistema de Exámenes Médicos.

Respondes consultas sobre:
- Información de alumnos
- Estado de exámenes médicos
- Reportes y estadísticas generales

Herramientas:
- consultar_alumno: Buscar por código, documento o apellido
- obtener_estado_examenes: Ver progreso de un alumno
- listar_alumnos_pendientes: Lista de pendientes
- generar_reporte: Reportes estadísticos

Responde de forma clara y profesional."""),
    ("human", "{messages}"),
])

agente_coordinador = create_react_agent(
    llm_coordinador,
    tools=[consultar_alumno, obtener_estado_examenes, listar_alumnos_pendientes, generar_reporte],
    checkpointer=memory_coordinador,
    prompt=prompt_coordinador
)

/tmp/ipython-input-2019450965.py:23: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agente_coordinador = create_react_agent(


In [ ]:
# AGENTE 2: PERSONAL MÉDICO
llm_medico = ChatOpenAI(temperature=0.3, model="gpt-4o-mini")
memory_medico = MemorySaver()

prompt_medico = ChatPromptTemplate.from_messages([
    ("system", """Eres el Agente Asistente para Personal Médico.

Ayudas a:
- Registrar resultados de exámenes
- Consultar información de pacientes
- Verificar estado de exámenes

Tipos de examen:
- laboratorio: Análisis de sangre
- psicologico: Evaluación psicológica
- odontologia: Examen dental
- rayos_x: Rayos X de tórax

Estados:
- completado: Examen finalizado sin problemas
- requiere_seguimiento: Necesita atención adicional

Herramientas:
- consultar_alumno: Buscar alumno
- obtener_estado_examenes: Ver historial
- registrar_examen: Registrar resultado

Siempre verifica que el alumno exista antes de registrar.
Responde en español de forma profesional."""),
    ("human", "{messages}"),
])

agente_medico = create_react_agent(
    llm_medico,
    tools=[consultar_alumno, obtener_estado_examenes, registrar_examen],
    checkpointer=memory_medico,
    prompt=prompt_medico
)

# AGENTE 3: ADMINISTRATIVO
llm_admin = ChatOpenAI(temperature=0.2, model="gpt-4o-mini")
memory_admin = MemorySaver()

prompt_admin = ChatPromptTemplate.from_messages([
    ("system", """Eres el Agente Asistente para Personal Administrativo.

Gestionas:
- Listas de alumnos pendientes
- Reportes estadísticos
- Consultas masivas por escuela

Herramientas:
- listar_alumnos_pendientes: Ver pendientes
- generar_reporte: Estadísticas
- consultar_alumno: Buscar información

Responde en español de forma organizada."""),
    ("human", "{messages}"),
])

agente_administrativo = create_react_agent(
    llm_admin,
    tools=[listar_alumnos_pendientes, generar_reporte, consultar_alumno],
    checkpointer=memory_admin,
    prompt=prompt_admin
)

/tmp/ipython-input-912427625.py:33: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agente_medico = create_react_agent(
/tmp/ipython-input-912427625.py:61: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agente_administrativo = create_react_agent(


# configurando conversacion

In [ ]:
# Variable global para mantener el contexto de conversación
SESION_ACTUAL = {
    "agente_tipo": "coordinador",
    "session_id": f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
}

In [ ]:
def conversar(consulta: str, agente_tipo: str = None) -> str:
    """
    Función principal para conversación fluida.
    Mantiene el contexto entre llamadas.

    Args:
        consulta: Pregunta o instrucción del usuario
        agente_tipo: 'coordinador', 'medico' o 'administrativo' (opcional)

    Returns:
        Respuesta del agente
    """
    global SESION_ACTUAL

    # Si no se especifica agente, usar el actual
    if agente_tipo is None:
        agente_tipo = SESION_ACTUAL["agente_tipo"]
    else:
        SESION_ACTUAL["agente_tipo"] = agente_tipo

    # Seleccionar agente
    agentes = {
        "coordinador": agente_coordinador,
        "medico": agente_medico,
        "administrativo": agente_administrativo
    }

    if agente_tipo not in agentes:
        return "Error: Agente no válido. Usa: 'coordinador', 'medico' o 'administrativo'"

    agente = agentes[agente_tipo]
    config = {"configurable": {"thread_id": SESION_ACTUAL["session_id"]}}

    # Ejecutar agente
    resultado = None
    for step in agente.stream(
        {"messages": [HumanMessage(content=consulta)]},
        config,
        stream_mode="values",
    ):
        mensaje = step["messages"][-1]
        if isinstance(mensaje, AIMessage):
            resultado = mensaje.content

    return resultado


In [ ]:
def cambiar_agente(nuevo_agente: str):
    """Cambia el agente activo"""
    global SESION_ACTUAL
    if nuevo_agente in ["coordinador", "medico", "administrativo"]:
        SESION_ACTUAL["agente_tipo"] = nuevo_agente
        return f"Agente cambiado a: {nuevo_agente}"
    return "Agente no válido"

def nueva_sesion():
    """Inicia una nueva sesión de conversación"""
    global SESION_ACTUAL
    SESION_ACTUAL = {
        "agente_tipo": "coordinador",
        "session_id": f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    }
    return "Nueva sesión iniciada"

FUNCIONES COMPLEMENTARIAS

In [ ]:
def info_sistema():
    """Muestra información del sistema"""
    return f"""
Sistema de Exámenes Médicos - Estado Actual
{'.'*60}
Agente activo: {SESION_ACTUAL['agente_tipo'].upper()}
Session ID: {SESION_ACTUAL['session_id']}
Archivo alumnos: {RUTA_EXCEL_ALUMNOS}
Archivo exámenes: {RUTA_EXCEL_EXAMENES}

Agentes disponibles:
- coordinador: Consultas generales, reportes
- medico: Registro de exámenes
- administrativo: Gestión masiva

Comandos:
- conversar("tu pregunta"): Consultar al agente actual
- cambiar_agente("tipo"): Cambiar de agente
- nueva_sesion(): Reiniciar conversación
- info_sistema(): Ver esta información
{'.'*60}
    """

def obtener_estadisticas_rapidas():
    """Obtiene estadísticas rápidas del sistema"""
    try:
        df_alumnos = gestor.cargar_alumnos()
        df_examenes = gestor.cargar_examenes()

        total_alumnos = len(df_alumnos)
        total_examenes_registrados = len(df_examenes)

        # Calcular completados
        codigos_unicos = df_examenes['CODIGO_MATRICULA'].unique()
        completos = 0

        for codigo in codigos_unicos:
            examenes_codigo = df_examenes[df_examenes['CODIGO_MATRICULA'] == codigo]
            if len(examenes_codigo[examenes_codigo['ESTADO'] == 'completado']) == 4:
                completos += 1

        return {
            "total_alumnos": total_alumnos,
            "total_examenes_registrados": total_examenes_registrados,
            "alumnos_proceso_completo": completos,
            "porcentaje_avance": round((completos / total_alumnos * 100) if total_alumnos > 0 else 0, 1)
        }
    except Exception as e:
        return {"error": str(e)}


exportacion de resultados

In [ ]:
def exportar_reporte_completo(ruta_salida: str = "/content/drive/MyDrive/Reporte_Examenes.xlsx"):
    """
    Exporta un reporte completo a Excel con múltiples hojas.
    """
    try:
        df_alumnos = gestor.cargar_alumnos()
        df_examenes = gestor.cargar_examenes()

        # Crear reporte consolidado
        reporte_consolidado = []

        for _, alumno in df_alumnos.iterrows():
            codigo = alumno['CODIGO MATRICULA']
            examenes = df_examenes[df_examenes['CODIGO_MATRICULA'].astype(str) == str(codigo)]

            # Contar completados
            completados = len(examenes[examenes['ESTADO'] == 'completado'])

            reporte_consolidado.append({
                'CODIGO': codigo,
                'NOMBRE_COMPLETO': f"{alumno['NOMBRES']} {alumno['APELLIDO PATERNO']} {alumno['APELLIDO MATERNO']}",
                'ESCUELA': alumno.get('NOMBRE ESCUELA', ''),
                'EXAMENES_COMPLETADOS': completados,
                'EXAMENES_PENDIENTES': 4 - completados,
                'PROGRESO_%': round((completados / 4) * 100, 1)
            })

        df_reporte = pd.DataFrame(reporte_consolidado)

        # Exportar a Excel con múltiples hojas
        with pd.ExcelWriter(ruta_salida, engine='openpyxl') as writer:
            df_reporte.to_excel(writer, sheet_name='Resumen', index=False)
            df_examenes.to_excel(writer, sheet_name='Examenes_Detalle', index=False)

            # Hoja de estadísticas
            stats = obtener_estadisticas_rapidas()
            df_stats = pd.DataFrame([stats])
            df_stats.to_excel(writer, sheet_name='Estadisticas', index=False)

        return f"Reporte exportado exitosamente a: {ruta_salida}"

    except Exception as e:
        return f"Error al exportar reporte: {str(e)}"

In [ ]:
def validar_archivos():
    """
    Valida que los archivos Excel existan y tengan el formato correcto.
    """
    validacion = {
        "archivo_alumnos": {
            "existe": False,
            "columnas_correctas": False,
            "total_registros": 0,
            "errores": []
        },
        "archivo_examenes": {
            "existe": False,
            "columnas_correctas": False,
            "total_registros": 0,
            "errores": []
        }
    }

    # Validar archivo de alumnos
    try:
        if Path(RUTA_EXCEL_ALUMNOS).exists():
            validacion["archivo_alumnos"]["existe"] = True
            df = gestor.cargar_alumnos()
            validacion["archivo_alumnos"]["total_registros"] = len(df)

            columnas_requeridas = [
                'CODIGO MATRICULA', 'TIPO DOCUMENTO', 'NUMERO DOCUMENTO',
                'APELLIDO PATERNO', 'APELLIDO MATERNO', 'NOMBRES',
                'FECHA NACIMIENTO', 'SEXO', 'CODIGO ESCUELA', 'NOMBRE ESCUELA'
            ]

            columnas_presentes = all(col in df.columns for col in columnas_requeridas)
            validacion["archivo_alumnos"]["columnas_correctas"] = columnas_presentes

            if not columnas_presentes:
                validacion["archivo_alumnos"]["errores"].append(
                    "Faltan columnas requeridas en el archivo"
                )
        else:
            validacion["archivo_alumnos"]["errores"].append(
                f"No se encontró el archivo en: {RUTA_EXCEL_ALUMNOS}"
            )
    except Exception as e:
        validacion["archivo_alumnos"]["errores"].append(str(e))

    # Validar archivo de exámenes
    try:
        if Path(RUTA_EXCEL_EXAMENES).exists():
            validacion["archivo_examenes"]["existe"] = True
            df = gestor.cargar_examenes()
            validacion["archivo_examenes"]["total_registros"] = len(df)
            validacion["archivo_examenes"]["columnas_correctas"] = True
        else:
            validacion["archivo_examenes"]["errores"].append(
                "Archivo de exámenes se creará automáticamente"
            )
    except Exception as e:
        validacion["archivo_examenes"]["errores"].append(str(e))

    return validacion

Ejemplo

**conversar(consulta: str, agente_tipo: str = None)**

AGENTES: Medico, administrativo

In [ ]:
conversar("Buscar el alumno con código 25010008")

'He encontrado la información del alumno con código de matrícula 25010008:\n\n- **Nombre:** Joselyn Atencio Aguilar\n- **Tipo de Documento:** D.N.I.\n- **Número de Documento:** 71483771\n- **Fecha de Nacimiento:** 12 de marzo de 2004\n- **Sexo:** Femenino\n- **Código de Escuela:** 110\n- **Nombre de Escuela:** Medicina Humana\n- **Año Base:** 2025\n\n### Estado de Exámenes Médicos:\n- **Exámenes Completados:** 1\n  - **Tipo de Examen:** Laboratorio\n  - **Fecha de Examen:** 26 de octubre de 2025\n  - **Resultado:** Resultados normales y fue evaluado por Dr. Pérez.\n  - **Estado:** Completado\n- **Exámenes Pendientes:** 3\n\nSi necesitas más información o detalles adicionales, no dudes en preguntar.'

In [ ]:
# Cambiar a agente médico
# Verificar estado de exámenes
cambiar_agente("medico")
conversar("¿Qué exámenes tiene registrados?")

'Por favor, proporcióname el código de matrícula, número de documento o apellido del alumno para poder consultar los exámenes registrados.'

In [ ]:
conversar("ok, su codigo es 25010008")


'La alumna **Joselyn Atencio Aguilar** con código de matrícula **25010008** tiene registrados los siguientes exámenes:\n\n- **Exámenes Completados:**\n  - **Tipo:** Laboratorio\n    - **Fecha de Examen:** 26 de octubre de 2025\n    - **Resultado:** Resultados normales y fue evaluado por Dr. Pérez.\n    - **Estado:** Completado\n\n- **Exámenes Pendientes:**\n  - **Tipo:** Psicológico\n    - **Nombre:** Evaluación Psicológica\n  - **Tipo:** Odontología\n    - **Nombre:** Examen Odontológico\n  - **Tipo:** Rayos X\n    - **Nombre:** Rayos X de Tórax\n\nEl progreso del proceso médico es del **25%** y hay un examen completado de un total de cuatro. Actualmente, no hay exámenes que requieran seguimiento. Si necesitas más información o realizar alguna acción, no dudes en decírmelo.'

In [ ]:
# Cambiar a agente médico: Cambio a agente médico para registro")
cambiar_agente("medico")


'Agente cambiado a: medico'

In [ ]:
# Registro y correccion  de examen
conversar("""Se tiene que hacer una correccion en el registro del examen de laboratorio para la primera alumna del Excel:
    - Resultados: Hemoglobina 14.5 g/dL, Glucosa 90 mg/dL, valores normales
    - Estado: completado
    - Responsable: Dr. Juan Pérez""")

/tmp/ipython-input-4020476754.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '25010008' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_examenes.loc[mask, col] = val
/tmp/ipython-input-4020476754.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_examenes.loc[mask, col] = val


'La corrección en el registro del examen de laboratorio para la alumna **Joselyn Atencio Aguilar** con código de matrícula **25010008** ha sido realizada con éxito. \n\n### Detalles del examen actualizado:\n- **Tipo de Examen:** Laboratorio\n- **Resultado:** Hemoglobina 14.5 g/dL, Glucosa 90 mg/dL, valores normales\n- **Estado:** Completado\n- **Responsable:** Dr. Juan Pérez\n\nSi necesitas realizar alguna otra acción o tienes más consultas, no dudes en decírmelo.'

In [ ]:
# Cambiar a administrativo
cambiar_agente("administrativo")
conversar("Dame un reporte general del estado actual")

'Aquí tienes el reporte general del estado actual:\n\n### Reporte General\n- **Fecha:** 26 de octubre de 2025\n- **Total de Alumnos:** 7,237\n- **Proceso Completo:** 0\n- **Porcentaje Completado:** 0.0%\n- **Requieren Seguimiento:** 0\n- **Exámenes Realizados por Tipo:**\n  - Laboratorio: 1\n\nSi necesitas más información o un reporte específico, no dudes en decírmelo.'

In [ ]:
# Lista de pendientes
conversar("Lista los 10  primeros alumnos que tienen exámenes pendientes")


'Aquí tienes el reporte general del estado actual:\n\n### Reporte General\n- **Fecha:** 26 de octubre de 2025\n- **Total de Alumnos:** 7,237\n- **Proceso Completo:** 0\n- **Porcentaje Completado:** 0.0%\n- **Requieren Seguimiento:** 0\n- **Exámenes Realizados por Tipo:**\n  - Laboratorio: 1\n\n---\n\n### Alumnos con Exámenes Pendientes (Primeros 10)\n1. **Código de Matrícula:** 25010257\n   - **Nombre Completo:** GABRIELA ALESSANDRA ACUÑA TORRES\n   - **Escuela:** MEDICINA HUMANA\n   - **Exámenes Pendientes:** Laboratorio, Psicológico, Odontología, Rayos X\n   - **Cantidad Pendientes:** 4\n\n2. **Código de Matrícula:** 25010258\n   - **Nombre Completo:** ANYER ALCANTARA CABREJOS\n   - **Escuela:** MEDICINA HUMANA\n   - **Exámenes Pendientes:** Laboratorio, Psicológico, Odontología, Rayos X\n   - **Cantidad Pendientes:** 4\n\n3. **Código de Matrícula:** 25010001\n   - **Nombre Completo:** CAMILA PATRICIA ALVARADO RISCO\n   - **Escuela:** MEDICINA HUMANA\n   - **Exámenes Pendientes:** La

In [ ]:
cambiar_agente("medico")
conversar("le falta pasar por todos los examenes entonces va a empezar con el examen de laboratorio en donde tiene resultados normales y fue evaluado por Dr. Pérez, registrar todo esto")

'Para registrar el examen de laboratorio, necesito el código de matrícula del alumno. Por favor, proporciónamelo para continuar.'

# ejemplo con otro alumno de codigo 25110777

In [ ]:
cambiar_agente("medico")
conversar('''Registra laboratorio para alumno 25110777:
Hemoglobina: 15.2 g/dL
Leucocitos: 7500/μL
Plaquetas: 250000/μL
Resultados normales
Estado: completado
Responsable: Dr. María Torres''')

'La alumna **Thais Bretha Gomez Asencios** con código de matrícula **25110777** tiene los siguientes exámenes pendientes:\n\n- **Evaluación Psicológica**\n- **Rayos X de Tórax** (ya registrado)\n\nHasta el momento, ha completado un total de **2 exámenes** de un total de **4**. Los exámenes completados son:\n\n1. **Laboratorio**\n   - Resultado: Hemoglobina: 15.2 g/dL, Leucocitos: 7500/μL, Plaquetas: 250000/μL, Resultados normales\n   - Estado: Completado\n   - Responsable: Dr. María Torres\n\n2. **Odontología**\n   - Resultado: Higiene dental adecuada, sin caries. Se recomienda limpieza dental preventiva\n   - Estado: Completado\n   - Responsable: Dr. Ana Sánchez\n\nSi necesitas realizar alguna acción adicional o tienes más consultas, no dudes en decírmelo.'

In [ ]:
conversar("si, claro. registra los resultados dados del alumno 25110777")

'El examen de laboratorio para la alumna **Thais Bretha Gomez Asencios** con código de matrícula **25110777** ha sido registrado con éxito. \n\n### Detalles del examen registrado:\n- **Tipo de Examen:** Laboratorio\n- **Resultado:** Hemoglobina: 15.2 g/dL, Leucocitos: 7500/μL, Plaquetas: 250000/μL, Resultados normales\n- **Estado:** Completado\n- **Responsable:** Dr. María Torres\n\nSi necesitas realizar alguna otra acción o tienes más consultas, no dudes en decírmelo.'

In [ ]:
conversar('''Registra odontología para alumno 25110777:
Higiene dental adecuada, sin caries
Se recomienda limpieza dental preventiva
Estado: completado
Responsable: Dr. Ana Sánchez''')

'El examen de odontología para la alumna **Thais Bretha Gomez Asencios** con código de matrícula **25110777** ha sido registrado con éxito. \n\n### Detalles del examen registrado:\n- **Tipo de Examen:** Examen Odontológico\n- **Resultado:** Higiene dental adecuada, sin caries. Se recomienda limpieza dental preventiva\n- **Estado:** Completado\n- **Responsable:** Dr. Ana Sánchez\n\nSi necesitas realizar alguna otra acción o tienes más consultas, no dudes en decírmelo.'

In [ ]:
conversar('''Registra rayos_x para alumno 25110777:
Radiografía de tórax sin alteraciones
Campos pulmonares normales
Estado: completado
Responsable: Dr. Podesta''')


'El examen de rayos X para la alumna **Thais Bretha Gomez Asencios** con código de matrícula **25110777** ha sido registrado con éxito. \n\n### Detalles del examen registrado:\n- **Tipo de Examen:** Rayos X de Tórax\n- **Resultado:** Radiografía de tórax sin alteraciones. Campos pulmonares normales\n- **Estado:** Completado\n- **Responsable:** Dr. Podesta\n\nActualmente, la alumna ha completado un total de **2 exámenes** de un total de **4**. Los exámenes pendientes son:\n- Evaluación Psicológica\n- Rayos X de Tórax (ya registrado)\n\nSi necesitas realizar alguna otra acción o tienes más consultas, no dudes en decírmelo.'

In [ ]:
conversar("al alumno 25110777 le falta pasar por algun examen?")

'La alumna **Thais Bretha Gomez Asencios** con código de matrícula **25110777** tiene los siguientes exámenes pendientes:\n\n- **Evaluación Psicológica**\n- **Rayos X de Tórax** (ya registrado)\n\nHasta el momento, ha completado un total de **2 exámenes** de un total de **4**. Los exámenes completados son:\n\n1. **Laboratorio**\n   - Resultado: Hemoglobina: 15.2 g/dL, Leucocitos: 7500/μL, Plaquetas: 250000/μL, Resultados normales\n   - Estado: Completado\n   - Responsable: Dr. María Torres\n\n2. **Odontología**\n   - Resultado: Higiene dental adecuada, sin caries. Se recomienda limpieza dental preventiva\n   - Estado: Completado\n   - Responsable: Dr. Ana Sánchez\n\nSi necesitas realizar alguna acción adicional o tienes más consultas, no dudes en decírmelo.'

In [ ]:
cambiar_agente("administrativo")
conversar("Genera reporte de los alumnos que hayan pasado por todos o al menos un examen mèdico")

'Aquí tienes el reporte general del estado actual y la lista de los primeros alumnos con exámenes pendientes:\n\n### Reporte General\n- **Fecha:** 26 de octubre de 2025\n- **Total de Alumnos:** 7,237\n- **Proceso Completo:** 0\n- **Porcentaje Completado:** 0.0%\n- **Requieren Seguimiento:** 0\n- **Exámenes Realizados por Tipo:**\n  - Laboratorio: 2\n  - Odontología: 1\n  - Rayos X: 1\n\n---\n\n### Alumnos con Exámenes Pendientes (Primeros 10)\n1. **Código de Matrícula:** 25010257\n   - **Nombre Completo:** GABRIELA ALESSANDRA ACUÑA TORRES\n   - **Escuela:** MEDICINA HUMANA\n   - **Exámenes Pendientes:** Laboratorio, Psicológico, Odontología, Rayos X\n   - **Cantidad Pendientes:** 4\n\n2. **Código de Matrícula:** 25010258\n   - **Nombre Completo:** ANYER ALCANTARA CABREJOS\n   - **Escuela:** MEDICINA HUMANA\n   - **Exámenes Pendientes:** Laboratorio, Psicológico, Odontología, Rayos X\n   - **Cantidad Pendientes:** 4\n\n3. **Código de Matrícula:** 25010001\n   - **Nombre Completo:** CAMI

In [ ]:
cambiar_agente("coordinador")
conversar("Busca todos los alumnos con apellido RIVERA")

'He encontrado la información de los alumnos con el apellido "RIVERA". A continuación, se detallan algunos de ellos:\n\n1. **Freddy Fabricio Arcos Rivera**\n   - **Código Matrícula:** 25010006\n   - **Tipo Documento:** D.N.I.\n   - **Número Documento:** 71937876\n   - **Fecha de Nacimiento:** 23 de febrero de 2006\n   - **Sexo:** Masculino\n   - **Nombre Escuela:** Medicina Humana\n   - **Exámenes Completados:** 0\n   - **Exámenes Pendientes:** 4\n\n2. **Nelly Virginia Rivera Villalobos**\n   - **Código Matrícula:** 25010344\n   - **Tipo Documento:** D.N.I.\n   - **Número Documento:** 73224408\n   - **Fecha de Nacimiento:** 11 de abril de 2002\n   - **Sexo:** Femenino\n   - **Nombre Escuela:** Medicina Humana\n   - **Exámenes Completados:** 0\n   - **Exámenes Pendientes:** 4\n\n3. **Luisa Fernanda Rivera Ordoñez**\n   - **Código Matrícula:** 25010399\n   - **Tipo Documento:** D.N.I.\n   - **Número Documento:** 75979536\n   - **Fecha de Nacimiento:** 25 de febrero de 2005\n   - **Sexo:*

In [ ]:
#conversar("...", agente_tipo="coordinador")